In [1]:
import copy
import numpy as np
import os

import pickle

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import time

from torch.autograd import Variable

# from extractor import get_features
# from extractor
os.listdir()

data_dir = 'data'
train_np_dir = 'train_np_subset'
test_np_dir = 'test_np_subset'
trained_weights_dir = 'trained_weights'

In [2]:
# Load existing clasess
pkl_file = open('classes_dict.pickle', 'rb')

classes_dict= pickle.load(pkl_file)

pkl_file.close()

print(classes_dict['PlayingGuitar'], len(classes_dict))

62 101


In [3]:
hidden_size = 128
input_size = 512
num_layers = 2
batch_size = 300
num_epochs = 100
learning_rate = 0.01
num_classes = len(classes_dict)
sequence_length = 50
# label_str_to_int = {'ApplyEyeMakeup': 0, 'Archery': 1, 'ApplyLipstick': 2}

### Define models

In [4]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN(input_size=512, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).cuda()
rnn


RNN (
  (lstm): LSTM(512, 128, num_layers=2, batch_first=True)
  (fc): Linear (128 -> 101)
)

In [5]:
# RNN Model (Many-to-One)
class GRU_rnn(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU_rnn, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

gru = GRU_rnn(input_size=512, hidden_size=256, num_layers=num_layers, num_classes=num_classes).cuda()
gru


GRU_rnn (
  (lstm): GRU(512, 256, num_layers=2, batch_first=True)
  (fc): Linear (256 -> 101)
)

 _____________________________________________________

In [6]:
use_gpu = torch.cuda.is_available()
print ("GPU is available: ", use_gpu)

GPU is available:  True


In [7]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [8]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=7):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer


In [32]:
def copute_dataset_size(path=''):
    data_classes = [i for i in os.listdir(path) if not i.startswith('.')]
    num_entries = 0

    for folder in data_classes:
        current_dir = path + '/' + folder + '/'
        num_entries += len(os.listdir(current_dir))
        
    return num_entries

train_size = copute_dataset_size(path=data_dir + '/' + train_np_dir)
test_size = copute_dataset_size(path=data_dir + '/' + test_np_dir)

def check_size(address, sequence_size=50):
    x = np.load(address)
    print (x.shape)
    if x.shape[0] < sequence_size:
        return np.concatenate((x, np.zeros((sequence_size - x.shape[0], x.shape[1]))), axis=0)
    return x

# Data Loader (Input Pipeline)
def get_loader(path=''):
    """
    Function reads .npy files from path.
    Returns:
        dataloader, data classes (list), size of input object [n_sequence, n_features], lenght_of_dataset
    """
    inputs = []
    targets = []
    data_classes = [i for i in os.listdir(path) if not i.startswith('.')]

    for folder in data_classes:
        current_dir = path + '/' + folder + '/'
        files = os.listdir(current_dir)
        #test_f = np.load(current_dir + files[0])[:,:30,:]
        
#         print(test_f.shape)
        temp = [torch.Tensor(np.load(current_dir +  f).reshape((sequence_length, input_size))) for f in files] 
                         # Transform to torch tensors
        
        targets += ([torch.LongTensor([classes_dict[folder]])] * len(temp))
        inputs += temp

    tensor_x = torch.stack(inputs)
    tensor_y = torch.stack(targets)
    my_dataset = torch.utils.data.TensorDataset(tensor_x, tensor_y)  # Create your datset
    my_dataloader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=True)  # Create your dataloader

    return (my_dataloader, data_classes)

dset_loaders = {x: get_loader(data_dir + '/' + x)[0] for x in [train_np_dir, test_np_dir]}

In [33]:
# os.listdir(data_dir + '/' + test_np_dir)

In [34]:
dset_sizes = {train_np_dir: train_size, test_np_dir: test_size}
dset_sizes

{'test_np_subset': 111, 'train_np_subset': 152}

In [35]:
def train_model(model, criterion, optimizer, lr_scheduler, path, num_epochs=200, model_name='lstm'):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in [train_np_dir, test_np_dir]:
            if phase == train_np_dir:
                optimizer = lr_scheduler(optimizer, epoch)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dset_loaders[phase]:
                # get the inputs
                
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
#                     print(inputs)
                    inputs = Variable(inputs.view(-1, sequence_length, input_size).cuda())
                    labels = Variable(labels.view(-1).cuda())                        
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                    
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
#                 print (inputs.size())
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
#                 print (labels, outputs)

                # backward + optimize only if in training phase
                if phase == train_np_dir:
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects / dset_sizes[phase]

            print('running corrects: ', running_corrects)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == test_np_dir and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model)
        # saving weights
        torch.save(model, data_dir + '/' + trained_weights_dir + "/" + model_name + '_' + str(epoch) + ".pt")

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model


In [36]:
model_lstm = train_model(rnn, criterion, optimizer, exp_lr_scheduler, '', num_epochs=num_epochs, model_name='lstm')

Epoch 0/99
----------
LR is set to 0.001
running corrects:  152
train_np_subset Loss: 0.0003 Acc: 1.0000
running corrects:  102
test_np_subset Loss: 0.0038 Acc: 0.9189


C:\Users\andre\Anaconda3\lib\site-packages\torch\serialization.py:147: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "



Epoch 1/99
----------
running corrects:  152
train_np_subset Loss: 0.0003 Acc: 1.0000
running corrects:  102
test_np_subset Loss: 0.0037 Acc: 0.9189

Epoch 2/99
----------
running corrects:  152
train_np_subset Loss: 0.0003 Acc: 1.0000
running corrects:  102
test_np_subset Loss: 0.0038 Acc: 0.9189

Epoch 3/99
----------
running corrects:  152
train_np_subset Loss: 0.0003 Acc: 1.0000
running corrects:  102
test_np_subset Loss: 0.0038 Acc: 0.9189

Epoch 4/99
----------
running corrects:  152
train_np_subset Loss: 0.0003 Acc: 1.0000
running corrects:  102
test_np_subset Loss: 0.0038 Acc: 0.9189

Epoch 5/99
----------
running corrects:  152
train_np_subset Loss: 0.0003 Acc: 1.0000
running corrects:  100
test_np_subset Loss: 0.0038 Acc: 0.9009

Epoch 6/99
----------
running corrects:  152
train_np_subset Loss: 0.0003 Acc: 1.0000
running corrects:  100
test_np_subset Loss: 0.0039 Acc: 0.9009

Epoch 7/99
----------
LR is set to 0.0001
running corrects:  152
train_np_subset Loss: 0.0002 Acc: 

Epoch 54/99
----------
running corrects:  152
train_np_subset Loss: 0.0002 Acc: 1.0000
running corrects:  99
test_np_subset Loss: 0.0039 Acc: 0.8919

Epoch 55/99
----------
running corrects:  152
train_np_subset Loss: 0.0002 Acc: 1.0000
running corrects:  99
test_np_subset Loss: 0.0039 Acc: 0.8919

Epoch 56/99
----------
LR is set to 1.0000000000000006e-11
running corrects:  152
train_np_subset Loss: 0.0002 Acc: 1.0000
running corrects:  99
test_np_subset Loss: 0.0039 Acc: 0.8919

Epoch 57/99
----------
running corrects:  152
train_np_subset Loss: 0.0002 Acc: 1.0000
running corrects:  99
test_np_subset Loss: 0.0039 Acc: 0.8919

Epoch 58/99
----------
running corrects:  152
train_np_subset Loss: 0.0002 Acc: 1.0000
running corrects:  99
test_np_subset Loss: 0.0039 Acc: 0.8919

Epoch 59/99
----------
running corrects:  152
train_np_subset Loss: 0.0002 Acc: 1.0000
running corrects:  99
test_np_subset Loss: 0.0039 Acc: 0.8919

Epoch 60/99
----------
running corrects:  152
train_np_subset Lo

In [47]:
# # Test set is not implemented yet

# # Test the Model
# correct = 0
# total = 0
# for images, labels in test_loader:
#     images = Variable(images.view(-1, sequence_length, input_size))
#     outputs = rnn(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum()

# print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# # Save the Model
# # torch.save(rnn.state_dict(), 'rnn.pkl')
# # torch.save(rnn, 'rnn.pt')

# Generate dummy data set

In [ ]:
import numpy as np

In [ ]:
n_feat = 100
n_seq = 50 # fixed for now
n_video = 10

In [ ]:
for i in range(4,5):
    tmp_data = np.random.random((n_seq+1, n_feat))
    np.save('data/features/Archery/v' + str(i) + '.npy', tmp_data)